<a href="https://colab.research.google.com/github/vatsasree/CV-Assignment-2/blob/main/CV_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets
from torch.utils.data import DataLoader
import cv2
import numpy as np


In [2]:
train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True,
    transform=torchvision.transforms.ToTensor()
    )

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True,
    transform=torchvision.transforms.ToTensor()
    )


100%|██████████| 9912422/9912422 [00:00<00:00, 111400612.64it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 45780685.50it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31391208.86it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3803259.89it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
print(train_set)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [4]:
print(test_set)

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [5]:
batch_size = 60
n_workers = 4

In [6]:
train_dataset = DataLoader(train_set,
                           shuffle=True)

In [7]:
test_dataset = DataLoader(test_set,
                           shuffle=True)

In [8]:
# def extract_sift_features(image):
#     # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     if image.dtype != np.uint8:
#         image = (image * 255).astype(np.uint8)
#     gray = image
#     print(gray.shape, type(gray))
#     sift = cv2.SIFT_create()
#     keypoints, descriptors = sift.detectAndCompute(gray, None)
#     return keypoints, descriptors

# from sklearn.cluster import KMeans

# def cluster_sift_features(descriptors, num_clusters):
#     kmeans = KMeans(n_clusters=num_clusters)
#     kmeans.fit(descriptors)
#     return kmeans.cluster_centers_

# def compute_histogram(keypoints, descriptors, codebook):
#     # kmeans = KMeans(n_clusters=codebook.shape[0], init=codebook)
#     kmeans = KMeans(n_clusters=num_clusters)
#     kmeans.fit(descriptors)
#     histogram = np.zeros(codebook.shape[0])
#     for cluster_label in kmeans.labels_:
#         histogram[cluster_label] += 1
#     return histogram

def compute_cluster_centers(descriptors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(descriptors)
    return kmeans.cluster_centers_

# Function to compute histogram of visual words for an image
def compute_histogram(descriptors, cluster_centers):
    histogram = np.zeros(len(cluster_centers))
    for descriptor in descriptors:
        distances = np.linalg.norm(cluster_centers - descriptor, axis=1)
        nearest_cluster = np.argmin(distances)
        histogram[nearest_cluster] += 1
    return histogram


In [9]:
# from sklearn import svm
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Extract SIFT features, compute histograms, and build dataset
# X = []
# y = []

# num_clusters = 100


# for image, labels in train_dataset:
#     print(image.shape)
#     label = labels.item()
#     image = image[0].permute(1, 2, 0).numpy()
#     print(image.shape, type(image))
#     # gray_image = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
#     # gray_image = image.astype(np.uint8)
#     # gray_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2BGR)
#     keypoints, descriptors = extract_sift_features(image)
#     print(len(keypoints), len(descriptors))
#     codebook = cluster_sift_features(descriptors, num_clusters)
#     histogram = compute_histogram(keypoints, descriptors, codebook)
#     X.append(histogram)
#     y.append(label)

# X = np.array(X)
# y = np.array(y)

# # Split dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train linear SVM model
# svm_model = svm.LinearSVC()
# svm_model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = svm_model.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)


In [27]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml

# Function to compute cluster centers using K-means
def compute_cluster_centers(descriptors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(descriptors)
    return kmeans.cluster_centers_

# Function to compute histogram of visual words for an image
def compute_histogram(descriptors, cluster_centers):
    histogram = np.zeros(len(cluster_centers))
    for descriptor in descriptors:
        distances = np.linalg.norm(cluster_centers - descriptor, axis=1)
        nearest_cluster = np.argmin(distances)
        histogram[nearest_cluster] += 1
    return histogram

# # Load MNIST dataset
# mnist = fetch_openml('mnist_784', version=1)
# X, y = mnist.data, mnist.target.astype(int)

def full(num_clusters):
    # Initialize SIFT
    X_train, y_train = train_set.data.numpy(), train_set.targets.numpy()

    sift = cv2.SIFT_create()

    # Initialize lists to store descriptors
    descriptors_list = []
    labels_list = []

    for i in range(len(X_train)):
        image = X_train[i]
        keypoints, descriptors = sift.detectAndCompute(image, None)
        # descriptors_list.append(descriptors)
        if descriptors is not None and descriptors.any():
            descriptors_list.append(descriptors)
            labels_list.append(y_train[i])  # Append corresponding label

    # descriptors_list = [descriptors for descriptors in descriptors_list if descriptors is not None and descriptors.any()]

    # Convert descriptors to a single numpy array
    if descriptors_list:
        descriptors_array = np.concatenate(descriptors_list, axis=0)

        # Choose the number of clusters for K-means
        num_clusters = num_clusters

        # Compute cluster centers using K-means
        cluster_centers = compute_cluster_centers(descriptors_array, num_clusters)

        # Compute histograms of visual words for all images
        t_histograms = []
        t_labels = []
        for descriptors, label in zip(descriptors_list, labels_list):
            histogram = compute_histogram(descriptors, cluster_centers)
            t_histograms.append(histogram)
            t_labels.append(label)

        # Convert histograms and labels to numpy arrays
        X_train = np.array(t_histograms)
        y_train = np.array(t_labels)

        # Create a Linear SVM model
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))

        # Train the SVM model
        svm_model.fit(X_train, y_train)
    else:
        print("No descriptors found. Unable to create SVM model.")

    # Assuming you have a separate test set, similar to the training set
    X_test, y_test = test_set.data.numpy(), test_set.targets.numpy()

    # Initialize lists to store test descriptors and labels
    test_descriptors_list = []
    test_labels_list = []

    for i in range(len(X_test)):
        image = X_test[i]
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None and descriptors.any():
            test_descriptors_list.append(descriptors)
            test_labels_list.append(y_test[i])

    # Compute histograms of visual words for all test images
    test_histograms = []
    test_labels = []
    for descriptors, label in zip(test_descriptors_list, test_labels_list):
        histogram = compute_histogram(descriptors, cluster_centers)
        test_histograms.append(histogram)
        test_labels.append(label)

    # Convert test histograms and labels to numpy arrays
    X_test = np.array(test_histograms)
    y_test = np.array(test_labels)

    # Make predictions using the trained SVM model
    y_pred = svm_model.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy of the SVM model on the test set:", accuracy)

    import pickle

    filename = 'finalized_model_{}.sav'.format(num_clusters)
    pickle.dump(svm_model, open(filename, 'wb'))


    return accuracy



In [28]:
clusters=[10,20,40,80,160,320]
accuracies=[]
for num_clusters in clusters:
  accuracy = full(num_clusters)
  accuracies.append(accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.3425245098039216


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.4172794117647059


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

Accuracy of the SVM model on the test set: 0.6387867647058824


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.7241625816993464


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.7976919934640523


In [24]:
# import pickle

# filename = 'finalized_model_2.sav'
# pickle.dump(svm_model, open(filename, 'wb'))



In [22]:
# # Assuming you have a separate test set, similar to the training set
# X_test, y_test = test_set.data.numpy(), test_set.targets.numpy()

# # Initialize lists to store test descriptors and labels
# test_descriptors_list = []
# test_labels_list = []

# for i in range(len(X_test)):
#     image = X_test[i]
#     keypoints, descriptors = sift.detectAndCompute(image, None)
#     if descriptors is not None and descriptors.any():
#         test_descriptors_list.append(descriptors)
#         test_labels_list.append(y_test[i])

# # Compute histograms of visual words for all test images
# test_histograms = []
# test_labels = []
# for descriptors, label in zip(test_descriptors_list, test_labels_list):
#     histogram = compute_histogram(descriptors, cluster_centers)
#     test_histograms.append(histogram)
#     test_labels.append(label)

# # Convert test histograms and labels to numpy arrays
# X_test = np.array(test_histograms)
# y_test = np.array(test_labels)

# # Make predictions using the trained SVM model
# y_pred = svm_model.predict(X_test)

# # Calculate the accuracy of the model
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy of the SVM model on the test set:", accuracy)
